In [1]:
import yaml
yaml.warnings({'YAMLLoadWarning': False})
import os

with open("./config.yaml", 'rb') as f:
    config = yaml.load(f)

In [2]:
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']

# load library

In [3]:
# python default library
import os
import shutil
import datetime
import sys
import pickle

# general analysis tool-kit
import numpy as np
import pandas as pd
import scipy
#from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# original library
sys.path.append('/home/hiroki/research/dcase2021_task2/src/functions')
import common as com
import pytorch_modeler as modeler

# etc
import yaml
yaml.warnings({'YAMLLoadWarning': False})

# ML lib
from scipy.stats import zscore
#from umap import UMAP
#from sklearn.mixture import GaussianMixture
from sklearn.covariance import LedoitWolf
from scipy.spatial.distance import mahalanobis


import librosa
import IPython
import librosa.display

# load config and set logger

In [4]:
log_folder = config['IO_OPTION']['OUTPUT_ROOT']+'/eval_strict_MVG_{0}.log'.format(datetime.date.today())
logger = com.setup_logger(log_folder, '01_eval.py')

# Setting

In [5]:
# Setting seed
modeler.set_seed(42)

In [6]:
############################################################################
# Setting I/O path
############################################################################
# input dirs
INPUT_ROOT = config['IO_OPTION']['INPUT_ROOT']
dev_dir = INPUT_ROOT + "/dev_data"
add_dev_dir = INPUT_ROOT + "/add_dev_data"
# machine type
machine_types = os.listdir(dev_dir)
# output dirs
OUTPUT_ROOT = config['IO_OPTION']['OUTPUT_ROOT']
MODEL_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/models'
TB_DIR = config['IO_OPTION']['OUTPUT_ROOT'] + '/tb'
OUT_FEATURE_DIR = OUTPUT_ROOT + '/extraction_features'
OUT_SCORE_DIR = OUTPUT_ROOT + '/score_strict'
OUT_PRED_DIR = OUTPUT_ROOT + '/pred_strict'
#os.makedirs(OUTPUT_ROOT, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(TB_DIR, exist_ok=True)
os.makedirs(OUT_FEATURE_DIR, exist_ok=True)
os.makedirs(OUT_SCORE_DIR, exist_ok=True)
os.makedirs(OUT_PRED_DIR, exist_ok=True)
# copy config
shutil.copy('./config.yaml', OUTPUT_ROOT)

'/media/hiroki/HDD1TB/research/dcase2021_task2/output/MahalanobisAD_SubSeq/frame64_3/config.yaml'

In [7]:
#score = pd.read_csv(f'{OUT_SCORE_DIR}/{machine_types[0]}_score.csv', index_col=0)

In [8]:
#score.loc['h_mean']['pAUC']

# load data

In [9]:
machine_types

['fan', 'gearbox', 'pump', 'slider', 'ToyCar', 'ToyTrain', 'valve']

In [10]:
data_types = ['train', 'valid_source', 'valid_target']

In [11]:
def load_ext_data(machine_type, phase):
    input_path = f'{OUT_FEATURE_DIR}/{machine_type}_{phase}_features.pkl'
    ext_data = pd.read_pickle(input_path)
    
    return ext_data

In [12]:
# for train datasets
def get_target_names(wav_names):
    target_names = []
    for wav_name in wav_names:
        if 'target' in wav_name:
            target_names.append(wav_name)
    
    return target_names

for machine_type in machine_types:
    data = load_ext_data(machine_type)
    plt.figure(figsize=(10,10))
    plt.imshow(data['train']['features'], aspect='auto')
    plt.title(machine_type)
    plt.colorbar()
    plt.show()

for machine_type in machine_types:
    data = load_ext_data(machine_type)
    plt.figure(figsize=(10,10))
    plt.imshow(data['valid_source']['features'], aspect='auto')
    plt.title(machine_type)
    plt.colorbar()
    plt.show()

for machine_type in machine_types:
    data = load_ext_data(machine_type)
    plt.figure(figsize=(10,10))
    plt.imshow(data['valid_target']['features'], aspect='auto')
    plt.title(machine_type)
    plt.colorbar()
    plt.show()

# evaluation

In [13]:
#ext_data = load_ext_data(machine_types[2], 'train')
#section_types = com.get_section_types(ext_data['train']['wav_names'])

- features
    - list(5000)
- wav_names
    - list(5000)
- label
    - list(5000)

In [14]:
out_dir = '/media/hiroki/HDD1TB/research/dcase2021_task2/output/MahalanobisAD_SubSeq/frame64_2/extraction_features_revised'

machine_types = machine_types[2:]

for machine_type in machine_types:
    print(machine_type)
    for phase in ['train', 'valid_source', 'valid_target']:
        print(phase)
        ext_data = load_ext_data(machine_type, phase)
        ext_data_revised = {}
        ext_data_revised['features'] = []
        ext_data_revised['label'] = []
        ext_data_revised['wav_name'] = []
        for i in range(len(ext_data)):
            ext_data_revised['features'].append(ext_data[i]['features'])
            ext_data_revised['label'].append(ext_data[i]['label'])
            ext_data_revised['wav_name'].append(ext_data[i]['wav_name'])
        del ext_data
        save_dir = f'{out_dir}/{machine_type}_{phase}_features.pkl'
        pd.to_pickle(ext_data_revised, save_dir)
        del ext_data_revised

## calc MVG (multivariate Gaussian)

In [17]:
MVG = {}
for machine_type in machine_types:
    print(f'load {machine_type}')
    MVG[machine_type] = {}
    ext_data = load_ext_data(machine_type, 'train')
    section_types = com.get_section_types(ext_data['wav_name'])
    
    for section in np.unique(section_types):
        print(f'calc {section}')
        MVG[machine_type][section] = {}
        idx = np.where(section_types == section)[0]
        per_section_samples = []
        for idx_ in idx:
            per_section_samples.append(ext_data['features'][idx_])
        per_section_samples = np.concatenate(per_section_samples, axis=0)
        per_section_mean = per_section_samples.mean(axis=0)
        I = np.identity(per_section_samples.shape[1])
        per_section_cov = np.cov(per_section_samples, rowvar=False) + 0.01 * I
        MVG[machine_type][section]['mean'] = per_section_mean
        MVG[machine_type][section]['cov'] = per_section_cov

load fan
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5
load gearbox
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5
load pump
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5
load slider
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5
load ToyCar
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5
load ToyTrain
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5
load valve
calc 0
calc 1
calc 2
calc 3
calc 4
calc 5


## estimate

In [18]:
from IPython.display import display

In [24]:
#pd.to_pickle(MVG, OUT_FEATURE_DIR+'/strict_MVG.pkl')

In [19]:
def get_score_perID(describe_df, max_fpr=0.1):
    # ユニークsectionを取得、昇順ソート
    sections = np.sort(describe_df['section_types'].unique())

    for section in sections:
        per_section_df = describe_df[describe_df['section_types'] == section]
        per_section_AUC = roc_auc_score(per_section_df['labels'], per_section_df['preds'])
        per_section_pAUC = roc_auc_score(per_section_df['labels'], per_section_df['preds'], max_fpr=max_fpr)
        # column = [AUC,pAUC], row = index
        score_df = pd.DataFrame(np.stack([per_section_AUC, per_section_pAUC]), index=['AUC', 'pAUC']).T
        # indexをsectionナンバーにrename
        # column = [AUC,pAUC], row = [section]
        score_df.index = [section]
        if section == 0:
            scores_df = score_df.copy()
        else:
            # 結合
            scores_df = scores_df.append(score_df)
    return scores_df

In [20]:
def calc_mahalanobis(mean, cov, samples):
    cov_inv = np.linalg.inv(cov)
    # load data
    dists = [mahalanobis(sample, mean, cov_inv) for sample in samples]
    # np.array
    dists = np.array(dists)
    return dists

In [29]:
def calc_anomaly_score(mean, cov_inv, sample, n_hop_frames = 1):
    sample = sample[:: n_hop_frames, :]
    # load data
    dists = [mahalanobis(sample[idx,:], mean, cov_inv) for idx in range(len(sample))]
    # np.array
    dists = np.array(dists)
    return dists.mean()

In [30]:
# load data
#dists = [mahalanobis(sample[idx,:], tr_mean, tr_cov_inv) for idx in range(len(sample))]
# np.array
#dists = np.array(dists)

## Calc Score

In [31]:
from tqdm import tqdm

In [32]:
# valid_dataloaderをシャッフルするとバグるの注意

for i, machine_type in enumerate(machine_types):
    logger.info('CALC SCORE')
    logger.info(machine_type)
    # get MVG
    #tr_mean = MVG[machine_type]['mean']
    #tr_cov = MVG[machine_type]['cov']
    # load samples
    for phase in ['valid_source', 'valid_target']:
        ext_data = load_ext_data(machine_type, phase)
        section_types = com.get_section_types(ext_data['wav_name']) # vector
        for section in np.unique(section_types): #(0,1,2,3,4,5)
            # get MVG
            tr_mean = MVG[machine_type][section]['mean']
            tr_cov_inv = MVG[machine_type][section]['cov']
            tr_cov_inv = np.linalg.inv(tr_cov_inv)
            idx = np.where(section_types == section)[0]
            # predict
            preds = []
            logger.info('Calc Anomaly Score')
            for idx_ in tqdm(range(len(idx))):
                sample = ext_data['features'][idx_] # 1 sample matrix
                # predict 1sample
                pred = calc_anomaly_score(tr_mean,
                                          tr_cov_inv,
                                          sample)
                preds.append(pred)
            preds = np.concatenate(preds)

            if section == np.unique(section_types)[0]:
                pred_all = preds.copy()
            else:
                pred_all = np.concatenate([pred_all, preds], axis=0)
        # wavname + pred
        preds_pd = np.stack([np.array(ext_data['wav_name']), pred_all], axis=1)
        preds_pd = pd.DataFrame(preds_pd, columns=['wav_name', 'pred'])
        preds_pd.to_csv(OUT_PRED_DIR + f'/pred_{machine_type}_{phase}.csv')
        # dataframe作成
        describe_df = com.get_pred_discribe(labels=ext_data['labels'],
                                            preds=preds,
                                            section_types=section_types)
        # スコア算出(AUC, pAUC)
        scores_df = com.get_score_per_Section(describe_df, max_fpr=0.1)

        # 結合(source + target)
        if phase == 'valid_source':
            scores_df = scores_df.rename(index=lambda num: 'Source_' + f'{num}')
            all_scores_df = scores_df.copy()
        else:
            scores_df = scores_df.rename(index=lambda num: 'Target_' + f'{num}')
            all_scores_df = all_scores_df.append(scores_df)
        del ext_data
            
    # 平均
    mean_df = pd.DataFrame(all_scores_df.mean(axis=0)).T
    mean_df.index = ['mean']
    # 調和平均
    hmean = scipy.stats.hmean(all_scores_df, axis=0)
    hmean_df = pd.DataFrame(hmean, index=['AUC', 'pAUC']).T
    hmean_df.index = ['h_mean']
    # 結合
    all_scores_df = all_scores_df.append(mean_df)
    all_scores_df = all_scores_df.append(hmean_df)
    # 出力
    all_scores_df.to_csv(f'{OUT_SCORE_DIR}/{machine_type}_score.csv')
    # display
    display(all_scores_df)

2021-05-26 00:53:32,105 - 01_eval.py - INFO - CALC SCORE
2021-05-26 00:53:32,106 - 01_eval.py - INFO - fan
2021-05-26 00:53:32,560 - 01_eval.py - INFO - Calc Anomaly Score
 40%|████      | 80/200 [00:47<01:10,  1.69it/s]


KeyboardInterrupt: 

In [43]:
pred_all.shape

(562800,)

In [41]:
preds_pd = np.stack([np.array(ext_data['wav_name']), pred_all], axis=1)
preds_pd = pd.DataFrame(preds_pd, columns=['wav_name', 'pred'])
preds_pd.to_csv(OUT_PRED_DIR + f'/pred_{machine_type}_{phase}.csv')
# dataframe作成
describe_df = com.get_pred_discribe(labels=ext_data['labels'],
                                    preds=preds,
                                    section_types=section_types)
# スコア算出(AUC, pAUC)
scores_df = com.get_score_per_Section(describe_df, max_fpr=0.1)

# 結合(source + target)
if phase == 'valid_source':
    scores_df = scores_df.rename(index=lambda num: 'Source_' + f'{num}')
    all_scores_df = scores_df.copy()
else:
    scores_df = scores_df.rename(index=lambda num: 'Target_' + f'{num}')
    all_scores_df = all_scores_df.append(scores_df)
del ext_data

ValueError: all input arrays must have the same shape

for machine_type in machine_types:
    # get MVG
    mean = MVG[machine_type]['mean']
    cov_inv = np.linalg.inv(MVG[machine_type]['cov'])
    # load data
    ext_data = load_ext_data(machine_type)
    # calc mahalanobis (Anomaly Score)
    valid_source_dists = [mahalanobis(sample, mean, cov_inv) for sample in ext_data['valid_source']['features']]
    valid_target_dists = [mahalanobis(sample, mean, cov_inv) for sample in ext_data['valid_target']['features']]
    # np.array
    valid_source_dists = np.array(valid_source_dists)
    valid_target_dists = np.array(valid_target_dists)
    # calc AUC
    roc_auc = roc_auc_score(ext_data['valid_source']['labels'], valid_source_dists)
    logger.info(f'{machine_type} valid_source AUC : {roc_auc}')
    roc_auc = roc_auc_score(ext_data['valid_target']['labels'], valid_target_dists)
    logger.info(f'{machine_type} valid_target AUC : {roc_auc}')
    
    plt.figure(figsize=(20,3))
    plt.title(f'Source {machine_type}')
    plt.plot(valid_source_dists, label='Anomaly Score')
    plt.plot(ext_data['valid_source']['labels'], label='Ground Truth')
    plt.legend()
    plt.show()
    
    plt.figure(figsize=(20,3))
    plt.title(f'Target {machine_type}')
    plt.plot(valid_target_dists, label='Anomaly Score')
    plt.plot(ext_data['valid_target']['labels'], label='Ground Truth')
    plt.legend()
    plt.show()

## calc GMM

In [ ]:
machine_types

In [ ]:
ext_data = load_ext_data(machine_types[6])
gmm = GaussianMixture(n_components=3, random_state=42)
gmm.fit(ext_data['train']['features'])

In [ ]:
gmm.bic(ext_data['train']['features'])

In [ ]:
pred = gmm.predict(ext_data['train']['features'])

In [ ]:
plt.hist(pred)

In [ ]:
gmm_covs = gmm.covariances_
gmm_means = gmm.means_

In [ ]:
gmm_covs = gmm.covariances_
gmm_means = gmm.means_


# calc mahalanobis (Anomaly Score)
valid_source_dists = [mahalanobis(sample, mean, cov_inv) for sample in ext_data['valid_source']['features']]
valid_target_dists = [mahalanobis(sample, mean, cov_inv) for sample in ext_data['valid_target']['features']]

In [ ]:
for machine_type in machine_types:
    # get MVG
    mean = MVG[machine_type]['mean']
    cov_inv = MVG[machine_type]['cov']
    # load data
    ext_data = load_ext_data(machine_type)
    # calc mahalanobis (Anomaly Score)
    valid_source_dists = [mahalanobis(sample, mean, cov_inv) for sample in ext_data['valid_source']['features']]
    valid_target_dists = [mahalanobis(sample, mean, cov_inv) for sample in ext_data['valid_target']['features']]
    # np.array
    valid_source_dists = np.array(valid_source_dists)
    valid_target_dists = np.array(valid_target_dists)
    # calc AUC
    roc_auc = roc_auc_score(ext_data['valid_source']['labels'], valid_source_dists)
    logger.info(f'{machine_type} valid_source AUC : {roc_auc}')
    roc_auc = roc_auc_score(ext_data['valid_target']['labels'], valid_target_dists)
    logger.info(f'{machine_type} valid_target AUC : {roc_auc}')

In [ ]:
gmm_section_types = gmm.predict(feats)

In [ ]:
sns.distplot(section_types)

In [ ]:
sns.displot(gmm_section_types)